# Basic Retirement Simulation

This notebook demonstrates the basic usage of FinSim for retirement planning.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Import FinSim modules
from finsim.simulation import SimulationConfig, RetirementSimulation
from finsim.mortality import calculate_survival_curve, calculate_life_expectancy

## 1. Basic Configuration

Let's start with a simple retirement scenario:

In [ ]:
# Create simulation configuration
config = SimulationConfig(
    # Demographics
    current_age=65,
    retirement_age=65,
    max_age=95,
    gender="Male",
    
    # Financial position
    initial_portfolio=1_000_000,
    
    # Annual spending & income
    annual_consumption=60_000,
    social_security=30_000,
    pension=0,
    
    # Market assumptions (real returns)
    expected_return=5.0,  # 5% real return
    return_volatility=16.0,  # 16% volatility
    dividend_yield=2.0,  # 2% dividends
    
    # Tax and simulation settings
    effective_tax_rate=15.0,
    n_simulations=1000,
    include_mortality=True
)

print(f"Net portfolio withdrawal needed: ${config.net_consumption_need:,.0f}/year")
print(f"Initial withdrawal rate: {config.net_consumption_need / config.initial_portfolio * 100:.2f}%")

## 2. Run Monte Carlo Simulation

In [ ]:
# Run simulation
sim = RetirementSimulation(config)
results = sim.run_monte_carlo()

print(f"Success rate: {results.success_rate:.1%}")
print(f"Median final portfolio: ${results.percentiles[50][-1]:,.0f}")
print(f"10th percentile final: ${results.percentiles[10][-1]:,.0f}")
print(f"90th percentile final: ${results.percentiles[90][-1]:,.0f}")

## 3. Visualize Portfolio Paths

In [ ]:
# Create portfolio projection chart
ages = np.arange(config.current_age, config.max_age + 1)

fig = go.Figure()

# Add percentile bands
fig.add_trace(go.Scatter(
    x=ages, y=results.percentiles[90],
    name='90th percentile',
    line=dict(color='lightgreen', width=1)
))

fig.add_trace(go.Scatter(
    x=ages, y=results.percentiles[75],
    fill='tonexty',
    name='75th percentile',
    line=dict(color='green', width=1),
    fillcolor='rgba(0,255,0,0.1)'
))

fig.add_trace(go.Scatter(
    x=ages, y=results.percentiles[50],
    name='Median',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x=ages, y=results.percentiles[25],
    fill='tonexty',
    name='25th percentile',
    line=dict(color='orange', width=1),
    fillcolor='rgba(255,165,0,0.1)'
))

fig.add_trace(go.Scatter(
    x=ages, y=results.percentiles[10],
    name='10th percentile',
    line=dict(color='red', width=1)
))

# Add zero line
fig.add_hline(y=0, line_dash="dash", line_color="red", annotation_text="Depletion")

fig.update_layout(
    title="Portfolio Value Projection (Real $)",
    xaxis_title="Age",
    yaxis_title="Portfolio Value ($)",
    height=500,
    hovermode='x unified'
)

fig.show()

## 4. Analyze Mortality Impact

In [ ]:
# Calculate life expectancy
life_exp = calculate_life_expectancy(config.current_age, config.gender)
print(f"Life expectancy at {config.current_age}: {life_exp:.1f} years")
print(f"Expected final age: {config.current_age + life_exp:.1f}")

# Calculate survival curve
survival_probs = calculate_survival_curve(
    config.current_age, 
    config.max_age, 
    config.gender
)

# Plot survival curve
fig_survival = go.Figure()
fig_survival.add_trace(go.Scatter(
    x=ages,
    y=survival_probs,
    mode='lines',
    name='Survival Probability'
))

fig_survival.update_layout(
    title=f"Survival Probability ({config.gender})",
    xaxis_title="Age",
    yaxis_title="Probability of Survival",
    yaxis_tickformat='.0%',
    height=400
)

fig_survival.show()

## 5. Failure Analysis

In [ ]:
# Analyze failures
failures = results.failure_years[results.failure_years <= 30]
if len(failures) > 0:
    failure_ages = config.current_age + failures
    
    fig_failure = go.Figure()
    fig_failure.add_trace(go.Histogram(
        x=failure_ages,
        nbinsx=20,
        name='Portfolio Depletion Age'
    ))
    
    fig_failure.update_layout(
        title="Distribution of Portfolio Failure Ages",
        xaxis_title="Age at Depletion",
        yaxis_title="Number of Scenarios",
        height=400
    )
    
    fig_failure.show()
    
    print(f"Failure rate: {len(failures) / config.n_simulations:.1%}")
    print(f"Median failure age: {np.median(failure_ages):.0f}")
    print(f"Earliest failure: Age {np.min(failure_ages):.0f}")
else:
    print("No failures in simulation!")

## 6. Sensitivity Analysis

Let's see how the success rate changes with different parameters:

In [ ]:
# Test different withdrawal rates
withdrawal_rates = [3.0, 3.5, 4.0, 4.5, 5.0]
success_rates = []

for rate in withdrawal_rates:
    test_config = SimulationConfig(
        current_age=65,
        max_age=95,
        initial_portfolio=1_000_000,
        annual_consumption=rate * 10_000 + 30_000,  # Withdrawal + SS
        social_security=30_000,
        expected_return=5.0,
        return_volatility=16.0,
        n_simulations=500  # Fewer for speed
    )
    
    test_sim = RetirementSimulation(test_config)
    test_results = test_sim.run_monte_carlo()
    success_rates.append(test_results.success_rate)

# Plot results
fig_sensitivity = go.Figure()
fig_sensitivity.add_trace(go.Scatter(
    x=withdrawal_rates,
    y=[s * 100 for s in success_rates],
    mode='lines+markers',
    name='Success Rate'
))

fig_sensitivity.add_hline(y=95, line_dash="dash", line_color="green", 
                         annotation_text="95% target")
fig_sensitivity.add_hline(y=80, line_dash="dash", line_color="orange", 
                         annotation_text="80% threshold")

fig_sensitivity.update_layout(
    title="Success Rate vs Withdrawal Rate",
    xaxis_title="Initial Withdrawal Rate (%)",
    yaxis_title="Success Rate (%)",
    height=400
)

fig_sensitivity.show()

# Print summary
for rate, success in zip(withdrawal_rates, success_rates):
    print(f"{rate}% withdrawal rate: {success:.1%} success rate")

## 7. Compare With and Without Mortality

Let's see how mortality risk affects the simulation:

In [ ]:
# Run with mortality
config_with_mortality = SimulationConfig(
    current_age=65,
    max_age=95,
    initial_portfolio=1_000_000,
    annual_consumption=60_000,
    social_security=30_000,
    expected_return=5.0,
    return_volatility=16.0,
    n_simulations=1000,
    include_mortality=True,
    gender="Male"
)

sim_with = RetirementSimulation(config_with_mortality)
results_with = sim_with.run_monte_carlo()

# Run without mortality
config_without_mortality = SimulationConfig(
    current_age=65,
    max_age=95,
    initial_portfolio=1_000_000,
    annual_consumption=60_000,
    social_security=30_000,
    expected_return=5.0,
    return_volatility=16.0,
    n_simulations=1000,
    include_mortality=False
)

sim_without = RetirementSimulation(config_without_mortality)
results_without = sim_without.run_monte_carlo()

print("Impact of Mortality Risk:")
print(f"Success rate WITH mortality: {results_with.success_rate:.1%}")
print(f"Success rate WITHOUT mortality: {results_without.success_rate:.1%}")
print(f"Difference: {(results_with.success_rate - results_without.success_rate)*100:.1f} percentage points")

# Note: Success rate typically INCREASES with mortality because 
# fewer scenarios require the portfolio to last the full 30 years

## Key Takeaways

1. **Withdrawal Rate Matters**: Higher withdrawal rates significantly reduce success probability
2. **Mortality Risk**: Including mortality typically increases success rates (portfolio doesn't need to last as long)
3. **Wide Outcome Range**: The 10th to 90th percentile range shows significant uncertainty
4. **Sequence Risk**: Early market downturns can cause portfolio failure even with reasonable withdrawal rates

## Next Steps

- Try different market assumptions (expected return, volatility)
- Add annuity income to see how guaranteed income affects outcomes
- Test different asset allocations or glide paths
- Incorporate more sophisticated tax strategies